In [4]:
import pdfplumber
import json
import re
import os

def extract_pages_to_json(input_pdf, output_json, start_page, end_page, section_title):
    with pdfplumber.open(input_pdf) as pdf:
        content = ""
        # 페이지 범위 조정: end_page가 start_page보다 작으면 start_page로 설정
        end_page = max(start_page, end_page)
        for page_num in range(start_page - 1, min(end_page, len(pdf.pages))):
            page = pdf.pages[page_num]
            content += page.extract_text(layout=True) + "\n\n"  # 레이아웃 보존
        
        # 기본적인 텍스트 정리
        content = re.sub(r'\n+', '\n', content).strip()
        
        # 내용이 비어있는지 확인
        if not content:
            print(f"Warning: No content extracted for section '{section_title}' (pages {start_page}-{end_page})")
            return
        
        # JSON 구조 생성
        section_data = {
            "title": section_title,
            "content": content,
            "start_page": start_page,
            "end_page": end_page
        }
        
        # JSON 파일로 저장
        with open(output_json, 'w', encoding='utf-8') as json_file:
            json.dump(section_data, json_file, ensure_ascii=False, indent=2)
        
        print(f"JSON file saved: {output_json}")


def main():
    input_pdf = '2-1. 수강신청_자료집_전체(2024-2).pdf'
    
    # JSON 파일을 저장할 디렉토리 생성
    output_dir = 'json_output'
    os.makedirs(output_dir, exist_ok=True)
    
    sections = [
        ('수강신청.json', 8, 18, "수강신청"),
        ('졸업이수학점 안내 (19년 이후 신입학자).json', 26, 31, "졸업이수학점 안내 (일반_편입생)"),
        ('졸업이수학점 안내 (공학인증학과, 19년 이후 신입학자).json', 44, 46, "졸업이수학점 안내 (공학인증학과)"),
        ('졸업이수학점 안내 (다전공, 19년 이후 신입학자).json', 37, 39, "졸업이수학점 안내 (다전공)"),
        ('교양 필수 교과목 수강 안내.json', 32, 33, "교양 필수 교과목 수강 안내"),
        ('편입생이 이수해야 할 교과목 및 학점.json', 34, 34, "편입생이 이수해야 할 교과목 및 학점"),
        ('다전공 안내.json', 34, 39, "다전공 안내"),
        ('연계전공 안내.json', 40, 41, "연계전공 안내"),
        ('공학교육인증제.json', 42, 46, "공학교육인증제"),
        ('학-석사 연계과정.json', 79, 80, "학-석사 연계과정"),
        ('서비스러닝 교과목 안내.json', 81, 82, "서비스러닝 교과목 안내"),
        ('다학년 다학기 프로젝트 (KW-VIP) 안내.json', 83, 86, "다학년 다학기 프로젝트 (KW-VIP) 안내"),
        ('참빛설계학기 안내.json', 87, 88, "참빛설계학기 안내"),
        ('K-MOOC 교내 온라인강좌 수강 안내.json', 89, 89, "K-MOOC 교내 온라인강좌 수강 안내"),
        ('매치업 집중이수제 교내 강좌 수강 안내.json', 90, 90, "매치업 집중이수제 교내 강좌 수강 안내"),
        ('현장실습학기제 안내.json', 91, 91, "현장실습학기제 안내"),
        ('특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제).json', 92, 95, "특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제)")
    ]
    
    for section in sections:
        output_json = os.path.join(output_dir, section[0])
        extract_pages_to_json(input_pdf, output_json, section[1], section[2], section[3])

if __name__ == "__main__":
    main()

JSON file saved: json_output\수강신청.json
JSON file saved: json_output\졸업이수학점 안내 (19년 이후 신입학자).json
JSON file saved: json_output\졸업이수학점 안내 (공학인증학과, 19년 이후 신입학자).json
JSON file saved: json_output\졸업이수학점 안내 (다전공, 19년 이후 신입학자).json
JSON file saved: json_output\교양 필수 교과목 수강 안내.json
JSON file saved: json_output\편입생이 이수해야 할 교과목 및 학점.json
JSON file saved: json_output\다전공 안내.json
JSON file saved: json_output\연계전공 안내.json
JSON file saved: json_output\공학교육인증제.json
JSON file saved: json_output\학-석사 연계과정.json
JSON file saved: json_output\서비스러닝 교과목 안내.json
JSON file saved: json_output\다학년 다학기 프로젝트 (KW-VIP) 안내.json
JSON file saved: json_output\참빛설계학기 안내.json
JSON file saved: json_output\K-MOOC 교내 온라인강좌 수강 안내.json
JSON file saved: json_output\매치업 집중이수제 교내 강좌 수강 안내.json
JSON file saved: json_output\현장실습학기제 안내.json
JSON file saved: json_output\특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제).json


In [5]:
import json
import re
import os

def detect_and_convert_table(text):
    lines = text.split('\n')
    if len(lines) < 2:  # 최소 2줄(헤더 + 데이터) 필요
        return text
    
    # 표 구분자 감지
    separators = re.findall(r'[-|]+', lines[1])
    if not separators:
        return text
    
    # 열 수 계산
    num_columns = len(separators[0].split('|'))
    
    # Markdown 표 생성
    md_table = []
    md_table.append('| ' + ' | '.join(lines[0].split()) + ' |')
    md_table.append('| ' + ' | '.join(['---' for _ in range(num_columns)]) + ' |')
    
    for line in lines[2:]:
        if not line.strip():  # 빈 줄 무시
            continue
        cells = line.split()
        if len(cells) != num_columns:  # 열 수가 맞지 않으면 표 종료
            break
        md_table.append('| ' + ' | '.join(cells) + ' |')
    
    return '\n'.join(md_table)

def json_to_markdown(json_file, md_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    markdown_content = f"# {data['title']}\n\n"
    
    # 내용을 단락으로 분리
    paragraphs = data['content'].split('\n\n')
    
    for para in paragraphs:
        # 표 감지 및 변환 시도
        converted_para = detect_and_convert_table(para)
        markdown_content += converted_para + '\n\n'
    
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

def main():
    json_dir = 'json_output'
    md_dir = 'markdown_output'
    os.makedirs(md_dir, exist_ok=True)
    
    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_file = os.path.join(json_dir, filename)
            md_file = os.path.join(md_dir, filename.replace('.json', '.md'))
            json_to_markdown(json_file, md_file)
            print(f"Converted {filename} to Markdown")

if __name__ == "__main__":
    main()

Converted K-MOOC 교내 온라인강좌 수강 안내.json to Markdown
Converted 공학교육인증제.json to Markdown
Converted 교양 필수 교과목 수강 안내.json to Markdown
Converted 다전공 안내.json to Markdown
Converted 다학년 다학기 프로젝트 (KW-VIP) 안내.json to Markdown
Converted 매치업 집중이수제 교내 강좌 수강 안내.json to Markdown
Converted 서비스러닝 교과목 안내.json to Markdown
Converted 수강신청.json to Markdown
Converted 연계전공 안내.json to Markdown
Converted 졸업이수학점 안내 (19년 이후 신입학자).json to Markdown
Converted 졸업이수학점 안내 (공학인증학과, 19년 이후 신입학자).json to Markdown
Converted 졸업이수학점 안내 (다전공, 19년 이후 신입학자).json to Markdown
Converted 참빛설계학기 안내.json to Markdown
Converted 특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제).json to Markdown
Converted 편입생이 이수해야 할 교과목 및 학점.json to Markdown
Converted 학-석사 연계과정.json to Markdown
Converted 현장실습학기제 안내.json to Markdown


In [1]:
import pdfplumber
import json
import re
import os

def extract_pages_to_json(input_pdf, output_json, start_page, end_page, section_title):
    with pdfplumber.open(input_pdf) as pdf:
        content = ""
        # 페이지 범위 조정: end_page가 start_page보다 작으면 start_page로 설정
        end_page = max(start_page, end_page)
        for page_num in range(start_page - 1, min(end_page, len(pdf.pages))):
            page = pdf.pages[page_num]
            content += page.extract_text(layout=True) + "\n\n"  # 레이아웃 보존
        
        # 기본적인 텍스트 정리
        content = re.sub(r'\n+', '\n', content).strip()
        
        # 내용이 비어있는지 확인
        if not content:
            print(f"Warning: No content extracted for section '{section_title}' (pages {start_page}-{end_page})")
            return
        
        # JSON 구조 생성
        section_data = {
            "title": section_title,
            "content": content,
            "start_page": start_page,
            "end_page": end_page
        }
        
        # JSON 파일로 저장
        with open(output_json, 'w', encoding='utf-8') as json_file:
            json.dump(section_data, json_file, ensure_ascii=False, indent=2)
        
        print(f"JSON file saved: {output_json}")


def main():
    input_pdf = './20240915190844072.pdf'
    
    # JSON 파일을 저장할 디렉토리 생성
    output_dir = 'json_output'
    os.makedirs(output_dir, exist_ok=True)
    
    sections = [
        ('1111.json', 1, 1, "1111"),
    ]
    
    for section in sections:
        output_json = os.path.join(output_dir, section[0])
        extract_pages_to_json(input_pdf, output_json, section[1], section[2], section[3])

if __name__ == "__main__":
    main()

JSON file saved: json_output\1111.json


In [2]:
import json
import re
import os

def detect_and_convert_table(text):
    lines = text.split('\n')
    if len(lines) < 2:  # 최소 2줄(헤더 + 데이터) 필요
        return text
    
    # 표 구분자 감지
    separators = re.findall(r'[-|]+', lines[1])
    if not separators:
        return text
    
    # 열 수 계산
    num_columns = len(separators[0].split('|'))
    
    # Markdown 표 생성
    md_table = []
    md_table.append('| ' + ' | '.join(lines[0].split()) + ' |')
    md_table.append('| ' + ' | '.join(['---' for _ in range(num_columns)]) + ' |')
    
    for line in lines[2:]:
        if not line.strip():  # 빈 줄 무시
            continue
        cells = line.split()
        if len(cells) != num_columns:  # 열 수가 맞지 않으면 표 종료
            break
        md_table.append('| ' + ' | '.join(cells) + ' |')
    
    return '\n'.join(md_table)

def json_to_markdown(json_file, md_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    markdown_content = f"# {data['title']}\n\n"
    
    # 내용을 단락으로 분리
    paragraphs = data['content'].split('\n\n')
    
    for para in paragraphs:
        # 표 감지 및 변환 시도
        converted_para = detect_and_convert_table(para)
        markdown_content += converted_para + '\n\n'
    
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

def main():
    json_dir = 'json_output'
    md_dir = 'markdown_output'
    os.makedirs(md_dir, exist_ok=True)
    
    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_file = os.path.join(json_dir, filename)
            md_file = os.path.join(md_dir, filename.replace('.json', '.md'))
            json_to_markdown(json_file, md_file)
            print(f"Converted {filename} to Markdown")

if __name__ == "__main__":
    main()

Converted 1111.json to Markdown
Converted K-MOOC 교내 온라인강좌 수강 안내.json to Markdown
Converted 공학교육인증제.json to Markdown
Converted 교양 필수 교과목 수강 안내.json to Markdown
Converted 다전공 안내.json to Markdown
Converted 다학년 다학기 프로젝트 (KW-VIP) 안내.json to Markdown
Converted 매치업 집중이수제 교내 강좌 수강 안내.json to Markdown
Converted 서비스러닝 교과목 안내.json to Markdown
Converted 수강신청.json to Markdown
Converted 연계전공 안내.json to Markdown
Converted 졸업이수학점 안내 (19년 이후 신입학자).json to Markdown
Converted 졸업이수학점 안내 (공학인증학과, 19년 이후 신입학자).json to Markdown
Converted 졸업이수학점 안내 (다전공, 19년 이후 신입학자).json to Markdown
Converted 참빛설계학기 안내.json to Markdown
Converted 특별교육과정 (CDP, R.O.T.C., 글로벌인재트랙인증제).json to Markdown
Converted 편입생이 이수해야 할 교과목 및 학점.json to Markdown
Converted 학-석사 연계과정.json to Markdown
Converted 현장실습학기제 안내.json to Markdown
